In [3]:
!pip install psycopg2-binary pandas

In [5]:
import sys
print(sys.version)

3.12.7 | packaged by Anaconda, Inc. | (main, Oct  4 2024, 13:17:27) [MSC v.1929 64 bit (AMD64)]


In [7]:
# Test PostgreSQL connection
import psycopg2

try:
    conn = psycopg2.connect(
        dbname="ngo_grants",
        user="postgres",
        password="shockwave16",  # Replace with your PostgreSQL password
        host="localhost",
        port="5432"  # Change to your confirmed port if not 5432
    )
    print("Connected to PostgreSQL!")
    conn.close()
except Exception as e:
    print(f"Error: {e}")

Connected to PostgreSQL!


In [9]:
# Create table
conn = psycopg2.connect(
    dbname="ngo_grants",
    user="postgres",
    password="shockwave16",
    host="localhost",
    port="5432"  # Change if your port differs
)
cursor = conn.cursor()

cursor.execute("""
    CREATE TABLE IF NOT EXISTS grants (
        id SERIAL PRIMARY KEY,
        funder_name TEXT NOT NULL,
        website TEXT NOT NULL,
        description TEXT NOT NULL,
        support_type VARCHAR(50) NOT NULL CHECK (support_type IN ('Grant', 'Donation', 'Contest')),
        support_amount TEXT,
        application_link TEXT NOT NULL,
        open_date DATE,
        close_date DATE,
        typical_grant_size TEXT,
        geographic_focus VARCHAR(100),
        target_demographic VARCHAR(100),
        project_category VARCHAR(100),
        ranking_score REAL CHECK (ranking_score >= 1 AND ranking_score <= 5),
        created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
    );
    CREATE INDEX IF NOT EXISTS idx_ranking_score ON grants (ranking_score);
    CREATE INDEX IF NOT EXISTS idx_close_date ON grants (close_date);
    CREATE INDEX IF NOT EXISTS idx_geographic_focus ON grants (geographic_focus);
""")
conn.commit()
print("Table and indexes created!")
cursor.close()
conn.close()

Table and indexes created!


In [11]:
conn = psycopg2.connect(
    dbname="ngo_grants",
    user="postgres",
    password="shockwave",
    host="localhost",
    port="5432"
)
cursor = conn.cursor()
cursor.execute("SELECT column_name, data_type FROM information_schema.columns WHERE table_name = 'grants';")
columns = cursor.fetchall()
for col in columns:
    print(col)
cursor.close()
conn.close()

('id', 'integer')
('funder_name', 'text')
('website', 'text')
('description', 'text')
('support_type', 'character varying')
('support_amount', 'text')
('application_link', 'text')
('open_date', 'date')
('close_date', 'date')
('typical_grant_size', 'text')
('geographic_focus', 'character varying')
('target_demographic', 'character varying')
('project_category', 'character varying')
('ranking_score', 'real')
('created_at', 'timestamp without time zone')


In [29]:
import psycopg2
import re

def calculate_ranking_score(description, support_amount, geographic_focus, project_category):
    # Mission Alignment (40%)
    mission_keywords = ['education', 'youth', 'africa', 'asia', 'schooling', 'training']
    mission_score = 3.0
    for keyword in mission_keywords:
        if keyword.lower() in description.lower() or keyword.lower() in project_category.lower():
            mission_score += 0.5
    if geographic_focus and geographic_focus.lower() in ['africa', 'asia']:
        mission_score += 0.5
    mission_score = min(mission_score, 5.0)

    # Funding Size (30%) - Updated logic
    funding_score = 3.0
    if support_amount:
        numbers = re.findall(r'\$?(\d+,\d+)', support_amount)
        if numbers:
            min_amount = int(numbers[0].replace(',', ''))
            max_amount = int(numbers[-1].replace(',', '')) if len(numbers) > 1 else min_amount
            if 10000 <= min_amount <= 100000 and max_amount <= 100000:
                funding_score = 5.0
            elif 10000 <= min_amount <= 100000 and max_amount <= 250000:
                funding_score = 4.6 
            elif (5000 <= min_amount < 10000) or (250000 < max_amount <= 500000):
                funding_score = 3.0
            else:
                funding_score = 1.0

    # Application Complexity (20%) - Heuristic
    complexity_score = 4.0

    # Success Likelihood (10%) - Heuristic
    likelihood_score = 4.0 if geographic_focus and geographic_focus.lower() in ['africa', 'asia'] else 3.0

    # Debug: Print individual scores
    print(f"Mission Score: {mission_score}, Funding Score: {funding_score}, "
          f"Complexity Score: {complexity_score}, Likelihood Score: {likelihood_score}")

    return round((0.4 * mission_score) + (0.3 * funding_score) + (0.2 * complexity_score) + (0.1 * likelihood_score), 1)

def insert_sample_grant():
    conn = psycopg2.connect(
        dbname="ngo_grants",
        user="postgres",
        password="shockwave16",
        host="localhost",
        port="5432"
    )
    cursor = conn.cursor()
    sample_grant = {
        'funder_name': 'Oak Foundation',
        'website': 'https://www.oakfnd.org',
        'description': 'Funds education projects for marginalized youth in East Africa, emphasizing access to quality schooling and teacher training.',
        'support_type': 'Grant',
        'support_amount': '$50,000-$200,000',
        'application_link': 'https://www.oakfnd.org/apply',
        'open_date': '2025-06-01',
        'close_date': '2025-07-31',
        'typical_grant_size': '$75,000-$150,000',
        'geographic_focus': 'Africa',
        'target_demographic': 'Youth',
        'project_category': 'Education',
        'ranking_score': calculate_ranking_score(
            description='Funds education projects for marginalized youth in East Africa, emphasizing access to quality schooling and teacher training.',
            support_amount='$50,000-$200,000',
            geographic_focus='Africa',
            project_category='Education'
        )
    }
    cursor.execute("""
        INSERT INTO grants (
            funder_name, website, description, support_type, support_amount, 
            application_link, open_date, close_date, typical_grant_size, 
            geographic_focus, target_demographic, project_category, ranking_score
        ) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
    """, (
        sample_grant['funder_name'], sample_grant['website'], sample_grant['description'],
        sample_grant['support_type'], sample_grant['support_amount'], sample_grant['application_link'],
        sample_grant['open_date'], sample_grant['close_date'], sample_grant['typical_grant_size'],
        sample_grant['geographic_focus'], sample_grant['target_demographic'], sample_grant['project_category'],
        sample_grant['ranking_score']
    ))
    conn.commit()
    cursor.close()
    conn.close()
    print("Sample grant inserted!")

# Clear previous entries to avoid duplicates
conn = psycopg2.connect(
    dbname="ngo_grants",
    user="postgres",
    password="shockwave16",
    host="localhost",
    port="5432"
)
cursor = conn.cursor()
cursor.execute("DELETE FROM grants;")
conn.commit()
cursor.close()
conn.close()

# Run insertion
insert_sample_grant()

# Verify
conn = psycopg2.connect(
    dbname="ngo_grants",
    user="postgres",
    password="shockwave16",
    host="localhost",
    port="5432"
)
cursor = conn.cursor()
cursor.execute("SELECT funder_name, ranking_score, geographic_focus FROM grants ORDER BY ranking_score DESC;")
results = cursor.fetchall()
for row in results:
    print(row)
cursor.close()
conn.close()

Mission Score: 5.0, Funding Score: 4.6, Complexity Score: 4.0, Likelihood Score: 4.0
Sample grant inserted!
('Oak Foundation', 4.6, 'Africa')


# Grant Opportunity Database Documentation

## Database Schema
- **Table**: `grants`
- **Fields**:
  - `id`: SERIAL PRIMARY KEY
  - `funder_name`: TEXT NOT NULL
  - `website`: TEXT NOT NULL
  - `description`: TEXT NOT NULL
  - `support_type`: VARCHAR(50) NOT NULL (Grant, Donation, Contest)
  - `support_amount`: TEXT
  - `application_link`: TEXT NOT NULL
  - `open_date`: DATE
  - `close_date`: DATE
  - `typical_grant_size`: TEXT
  - `geographic_focus`: VARCHAR(100)
  - `target_demographic`: VARCHAR(100)
  - `project_category`: VARCHAR(100)
  - `ranking_score`: REAL (1–5)
  - `created_at`: TIMESTAMP
- **Indexes**: `ranking_score`, `close_date`, `geographic_focus`
- **Scalability**: PostgreSQL supports large datasets; indexes optimize queries.

## Ranking System
- **Criteria**:
  - Mission Alignment (40%): Matches education/youth in Africa/Asia.
  - Funding Size (30%): Prefers $10,000–$100,000; close matches ($10,000–$250,000) score 4.6.
  - Application Complexity (20%): Ease of applying.
  - Success Likelihood (10%): Chance of securing.
- **Formula**: `Ranking Score = (0.4 * Mission_Score) + (0.3 * Funding_Score) + (0.2 * Complexity_Score) + (0.1 * Likelihood_Score)`
- **Automation**: Keyword matching and regex for scoring.

## Sample Entry
- **Funder**: Oak Foundation
- **Score**: 4.6 (Mission: 5.0, Funding: 4.6, Complexity: 4.0, Likelihood: 4.0)
- **Details**: Education projects in East Africa, $50,000–$200,000.

## Next Steps
- Share with client for approval.
- Adjust based on feedback.
- Proceed to Step 3 (automation).